In [31]:
import pandas as pd
import numpy as np
import vowpalwabbit

vw = vowpalwabbit.Workspace("--cb 4", quiet=True)



In [32]:
def parse_and_split_data(filename, test_ratio=0.2):
    rows = []

    with open(filename, 'r') as file:
        for line in file:
            if not line.strip():
                continue

            try:
                # Split action, cost, probability from features
                left, right = line.split('|')
                action_str, cost_str, prob_str = left.strip().split()

                row = {
                    'action': int(action_str),
                    'cost': float(cost_str),
                    'probability': float(prob_str),
                }

                features = right.strip().split()
                for feature in features:
                    key, value = feature.split('=')
                    row[key] = value

                rows.append(row)
            except ValueError as e:
                print(f"Skipping malformed line: {line.strip()} ({e})")
    split_idx = int(len(rows) * (1 - test_ratio))
    train_rows = rows[:split_idx]
    test_rows = rows[split_idx:]

    train_df = pd.DataFrame(train_rows).reset_index(drop=True)
    test_df = pd.DataFrame(test_rows).drop(columns=['action', 'cost', 'probability'], errors='ignore').reset_index(drop=True)


    return train_df, test_df



In [33]:
train_df, test_df = parse_and_split_data('cbdata.txt')

In [34]:
test_df.head()

,time_of_day,future_duedate,workload
0,19-20,high,high
1,19-20,low,low
2,13-14,high,high
3,12-13,low,high
4,21-22,low,medium


In [35]:
for i in train_df.index:
    action = train_df.loc[i, "action"]
    cost = train_df.loc[i, "cost"]
    probability = train_df.loc[i, "probability"]
    feature1 = train_df.loc[i, "time_of_day"]
    feature2 = train_df.loc[i, "future_duedate"]
    feature3 = train_df.loc[i, "workload"]

    # Construct the example in the required vw format.
    learn_example = (
        str(action)
        + ":"
        + str(cost)
        + ":"
        + str(probability)
        + " | "
        + str(feature1)
        + " "
        + str(feature2)
        + " "
        + str(feature3)
    )

    # Here we do the actual learning.
    vw.learn(learn_example)

In [36]:
for j in test_df.index:
    feature1 = test_df.loc[j, "time_of_day"]
    feature2 = test_df.loc[j, "future_duedate"]
    feature3 = test_df.loc[j, "workload"]

    test_example = "| " + str(feature1) + " " + str(feature2) + " " + str(feature3)

    choice = vw.predict(test_example)
    print(j, choice)

test_df.head()

0 1
1 2
2 1
3 4
4 4
5 1
6 2
7 4
8 2
9 2
10 4
11 4
12 4
13 1
14 2
15 3
16 4
17 1
18 4
19 4
20 4
21 2
22 1
23 4
24 4
25 4
26 4
27 4
28 4
29 1
30 1
31 4
32 3
33 4
34 4
35 4
36 4
37 2
38 4
39 2
40 4
41 1
42 4
43 4
44 4
45 4
46 4
47 4
48 2
49 4
50 4
51 4
52 1
53 2
54 1
55 2
56 2
57 4
58 4
59 1
60 4
61 1
62 4
63 4
64 4
65 1
66 3
67 3
68 4
69 2
70 4
71 1
72 3
73 1
74 2
75 3
76 1
77 4
78 2
79 2
80 4
81 4
82 4
83 4
84 3
85 1
86 4
87 4
88 4
89 1
90 3
91 4
92 2
93 4
94 2
95 4
96 4
97 4
98 4
99 1


,time_of_day,future_duedate,workload
0,19-20,high,high
1,19-20,low,low
2,13-14,high,high
3,12-13,low,high
4,21-22,low,medium


In [38]:
vw.save("cb.model")
del vw

vw = vowpalwabbit.Workspace("--cb 4 -i cb.model", quiet=True)
print(vw.predict("| a b"))

1
